Partie 1 : Scrapping

In [ ]:

import os
print("Emplacement Porjet_Web_Mining =", os.getcwd()) # affiche le dossier depuis lequel le script est lancé
print("data exists here? ->", os.path.isdir("data"))
print("data exists one level up? ->", os.path.isdir("../data"))


PWD = c:\Users\Violaine\OneDrive\Ecole\Master 1\Web mining\Projet Web Mining\Projet_Web_Mining\notebooks
data exists here? -> True
data exists one level up? -> False


In [ ]:
#Préparation des dossiers (évite erreurs “file not found”)
os.chdir("..")  # remonte de notebooks/ vers la racine du projet
os.makedirs("data", exist_ok=True)  # crée /data si pas déjà existant
os.makedirs("notebooks", exist_ok=True)  # crée /notebooks si pas déjà existant


In [ ]:
#Import des modules necessaires
import os # gestion de dossiers/fichiers
import re  # pour nettoyer un peu le texte (regex)
import time  # pour temporiser les requêtes (politesse + éviter blocages)
import random  # pour varier un peu les pauses (moins "robot")
import json  # pour stocker des listes (liens) proprement dans un CSV
from urllib.parse import urljoin, urlparse  # pour gérer URL relatives + domaines

import requests  # c'est ça qui va faire les requêtes HTTP (scraping)
from bs4 import BeautifulSoup  # pour parser le HTML
import pandas as pd  # pour stocker et exporter nos données
import numpy as np  # utile plus tard (matrices / graphes)


In [11]:
#paramètre de dépard
HEADERS = {  # on met un User-Agent classique, sinon certains sites renvoient 403 --> on fait croire qu'on est un vrai navigateur
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0 Safari/537.36"
}

REQUEST_TIMEOUT = 15  # si le site ne répond pas en 15 sec, on abandonne (évite de bloquer tout le notebook)
SLEEP_MIN = 1.0  # pause min entre 2 pages
SLEEP_MAX = 2.5  # pause max entre 2 pages

MAX_ARTICLES_PER_BLOG = 20  # on limite pour garder un corpus raisonnable (et pas scraper 2000 pages)
MAX_LINKS_PER_ARTICLE = 30  # limiter liens sortants => graphe plus propre + Gephi plus lisible

OUTPUT_ARTICLES = "data/articles.csv"  # notre corpus texte + métadonnées
OUTPUT_GEPHI_NODES = "data/gephi_nodes.csv"  # noeuds pour Gephi
OUTPUT_GEPHI_EDGES = "data/gephi_edges.csv"  # arêtes pour Gephi


In [5]:
# blogs de départ (à modifier si necessaire)
SEED_BLOGS = [
    "https://www.apartmenttherapy.com/",   # lifestyle / maison
    "https://www.theeverygirl.com/",       # lifestyle / self-care / carrière
    "https://cupcakesandcashmere.com/",    # lifestyle / mode
    "https://www.goop.com/",               # lifestyle / bien-être
    "https://www.thespruce.com/",          # maison / lifestyle
    "https://www.mindbodygreen.com/",      # bien-être / lifestyle
]


In [6]:
#fonction qu'on utilise 
def polite_sleep():  # petite pause entre requêtes
    time.sleep(random.uniform(SLEEP_MIN, SLEEP_MAX))  # pause aléatoire --> pour parraitre plus humain et moin rorbot

def is_valid_url(url: str) -> bool:  # filtre basique de liens inutiles
    if not url:  # vide
        return False
    if url.startswith(("mailto:", "tel:", "#")):  # email / tel / ancre
        return False
    return True

def normalize_url(url: str) -> str:  # normaliser un peu pour éviter doublons
    url = url.strip()  # enlever espaces
    url = url.split("#")[0]  # enlever ancres
    if url.endswith("/"):  # enlever / final (ça évite de compter 2 fois la même page)
        url = url[:-1]
    return url

def get_domain(url: str) -> str:  # récupérer le domaine
    return urlparse(url).netloc.lower()

def fetch_html(url: str) -> str | None:
    # ici on met requests.get dans une fonction, c'est plus clair que de répéter 20 fois la même ligne
    try:
        r = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT)  # requête HTTP
        if r.status_code != 200:  # si pas OK
            return None
        return r.text  # HTML brut
    except requests.RequestException:
        return None

def extract_links(base_url: str, soup: BeautifulSoup) -> list[str]:
    links = []
    for a in soup.find_all("a"):  # tous les liens <a href="...">
        href = a.get("href")
        if not is_valid_url(href):
            continue
        full = urljoin(base_url, href)  # transforme les liens relatifs en liens complets
        full = normalize_url(full)
        links.append(full)
    return links

def clean_text(text: str) -> str:
    # ici on fait simple : enlever multi-espaces + trim (espace de début et de fin)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def extract_main_text(soup: BeautifulSoup) -> str:
    # on enlève les scripts et styles car ça pollue fort le texte
    for tag in soup(["script", "style", "noscript"]):
        tag.decompose()
    return clean_text(soup.get_text(separator=" "))


In [7]:
#reperer un article ou un blog grace a l'oeuristique
def looks_like_article(url: str) -> bool:
    u = url.lower()
    patterns = ["blog", "post", "posts", "article", "story", "stories"] # mots courants dans les URL d'articles de blog
    if any(p in u for p in patterns):
        return True
    if re.search(r"/\d{4}/\d{2}/", u):  # format /2025/09/ souvent utilisé dans les blogs
        return True
    return False


In [ ]:
# Scraping d'un blog pour extraire articles
def collect_blog_articles(seed_url: str) -> list[dict]:  # fonction principale : récupère des articles à partir d'un blog "seed"
    
    seed_url = normalize_url(seed_url)  # on normalise l'URL (évite doublons du type / à la fin)
    seed_domain = get_domain(seed_url)  # on récupère le domaine (ex: mindbodygreen.com)

    html = fetch_html(seed_url)  # on télécharge le HTML de la page d'accueil du blog
    if html is None:  # si on n'a pas réussi à télécharger la page
        return []  # on renvoie une liste vide (pas d'article)

    soup = BeautifulSoup(html, "html.parser")  # on transforme le HTML en objet BeautifulSoup (plus facile à lire)

    all_links = extract_links(seed_url, soup)  # on récupère tous les liens présents sur la page d'accueil
    internal_links = []  # on va stocker uniquement les liens internes (même domaine)

    for link in all_links:  # on parcourt tous les liens trouvés
        if get_domain(link) == seed_domain:  # si le lien appartient au même domaine que le blog
            internal_links.append(link)  # alors c'est un lien interne, on le garde

    candidate_links = []  # ici on va garder les liens qui ressemblent à des articles
    for link in internal_links:  # on parcourt les liens internes
        if looks_like_article(link):  # si l'URL ressemble à un article (heuristique simple)
            candidate_links.append(link)  # on l'ajoute à la liste des candidats

    # enlever les doublons, sans casser l'ordre
    candidate_links = list(dict.fromkeys(candidate_links))  # astuce simple: dict garde l'ordre et supprime doublons

    articles = []  # liste finale qui contiendra les articles récupérés (en dict)

    for url in candidate_links[:MAX_ARTICLES_PER_BLOG]:  # on limite le nombre d'articles par blog (évite scrape trop large)
        
        polite_sleep()  # pause entre deux requêtes (évite d'être bloqué + respect des sites)
        
        article_html = fetch_html(url)  # on télécharge le HTML de la page candidate
        if article_html is None:  # si la page ne répond pas ou erreur
            continue  # on passe à l'URL suivante

        article_soup = BeautifulSoup(article_html, "html.parser")  # parser HTML de l'article

        title_tag = article_soup.find("title")  # on récupère la balise <title> (souvent le titre de la page)
        if title_tag:  # si la balise existe
            title = title_tag.get_text(strip=True)  # on récupère le texte du titre en enlevant les espaces
        else:  # si pas de titre
            title = ""  # on met vide

        text = extract_main_text(article_soup)  # on extrait le texte principal (sans scripts/styles)
        
        if len(text) < 500:  # si le texte est très court, c'est souvent une page "annexe" (menu/cookie/etc.)
            continue  # on l'ignore

        out_links = extract_links(url, article_soup)  # on récupère les liens présents dans l'article
        out_links = out_links[:MAX_LINKS_PER_ARTICLE]  # on limite le nombre de liens sortants (sinon graphe énorme)

        article_data = {  # on prépare un dictionnaire avec les infos utiles pour notre projet
            "seed_blog": seed_url,  # URL du blog de départ
            "seed_domain": seed_domain,  # domaine du blog (représente le blog dans le graphe)
            "url": url,  # URL de l'article
            "title": title,  # titre de l'article
            "text": text,  # texte brut (servira pour le text mining)
            "out_links": json.dumps(out_links),  # on stocke la liste en JSON pour la sauver dans un CSV
        }  # fin dict

        articles.append(article_data)  # on ajoute cet article à la liste finale

    return articles  # on renvoie la liste de tous les articles récupérés pour ce blog


In [12]:
# lancer la collecte sur tout les blogs
all_articles = []  # liste globale où on mettra tous les articles de tous les blogs

for seed in SEED_BLOGS:  # on parcourt chaque blog seed
    
    print("Collecting from:", seed)  # petit affichage pour suivre l'avancement
    
    data = collect_blog_articles(seed)  # on récupère les articles de ce blog
    
    print("  ->", len(data), "articles collected")  # on affiche combien d'articles on a réussi à récupérer
    
    all_articles.extend(data)  # on ajoute tous les articles collectés à la liste globale

df_articles = pd.DataFrame(all_articles)  # transformer la liste de dictionnaires en DataFrame pandas

df_articles.drop_duplicates(subset=["url"], inplace=True)  # enlever les doublons d'URL si jamais

print("Total articles:", len(df_articles))  # afficher le total final d'articles collectés

df_articles.head(5)  # afficher un aperçu des 5 premières lignes (utile pour vérifier)


  -> 0 articles collected
  -> 0 articles collected
  -> 1 articles collected
  -> 0 articles collected
  -> 0 articles collected
  -> 15 articles collected
Total articles: 16


,seed_blog,seed_domain,url,title,text,out_links
0,https://cupcakesandcashmere.com,cupcakesandcashmere.com,https://cupcakesandcashmere.com/blog,Home - Cupcakes & Cashmere,Home - Cupcakes & Cashmere Skip to content Fas...,"[""https://www.facebook.com/cupcakesandcashmere..."
1,https://www.mindbodygreen.com,www.mindbodygreen.com,https://www.mindbodygreen.com/articles/truth-a...,"The Truth About Lean Muscle, GLP-1s & Women’s ...","The Truth About Lean Muscle, GLP-1s & Women’s ...","[""https://www.mindbodygreen.com/accessibility""..."
2,https://www.mindbodygreen.com,www.mindbodygreen.com,https://www.mindbodygreen.com/articles/altras-...,Altra’s Experience Flow 2 Is A Standout For Th...,Altra’s Experience Flow 2 Is A Standout For Th...,"[""https://www.mindbodygreen.com/accessibility""..."
3,https://www.mindbodygreen.com,www.mindbodygreen.com,https://www.mindbodygreen.com/articles/what-to...,What The Toned Arms Aesthetic Really Means For...,What The Toned Arms Aesthetic Really Means For...,"[""https://www.mindbodygreen.com/accessibility""..."
4,https://www.mindbodygreen.com,www.mindbodygreen.com,https://www.mindbodygreen.com/articles/these-n...,These Natural Ingredients Could Gently Support...,These Natural Ingredients Could Gently Support...,"[""https://www.mindbodygreen.com/accessibility""..."


In [13]:
# sauvegarde du corpus en CSV
df_articles.to_csv(OUTPUT_ARTICLES, index=False, encoding="utf-8")  # on sauve le corpus, sans la colonne index
print("Saved:", OUTPUT_ARTICLES)  # confirmation dans la console


Saved: data/articles.csv


In [14]:
#Construire fichiers Gephi (nodes + edges)
nodes = set()  # set = évite les doublons automatiquement (chaque noeud une seule fois)
edges = []  # liste des liens (arêtes) entre noeuds

for _, row in df_articles.iterrows():  # on parcourt chaque ligne (article) du DataFrame
    
    source = row["seed_domain"]  # le noeud "source" = domaine du blog d'origine
    
    nodes.add(source)  # on ajoute la source dans la liste des noeuds
    
    out_links = json.loads(row["out_links"])  # on re-transforme la chaîne JSON en vraie liste python
    
    for link in out_links:  # pour chaque lien sortant trouvé dans l'article
        
        target = get_domain(link)  # le noeud "cible" = domaine du lien sortant
        
        if target == "":  # si le domaine est vide (rare mais possible)
            continue  # on ignore
        
        nodes.add(target)  # on ajoute aussi le noeud cible
        
        edge = {  # on crée une arête Gephi
            "Source": source,  # colonne Source attendue par Gephi
            "Target": target,  # colonne Target attendue par Gephi
            "Type": "Directed",  # graphe dirigé (utile pour PageRank / HITS)
        }  # fin dict
        
        edges.append(edge)  # on ajoute l'arête à la liste

df_nodes = pd.DataFrame(sorted(list(nodes)), columns=["Id"])  # Gephi aime une colonne "Id" pour les noeuds
df_nodes["Label"] = df_nodes["Id"]  # on met Label = Id pour simplifier

df_edges = pd.DataFrame(edges)  # on transforme la liste d'arêtes en DataFrame

df_nodes.to_csv(OUTPUT_GEPHI_NODES, index=False, encoding="utf-8")  # export nodes
df_edges.to_csv(OUTPUT_GEPHI_EDGES, index=False, encoding="utf-8")  # export edges

print("Saved:", OUTPUT_GEPHI_NODES)  # confirmation
print("Saved:", OUTPUT_GEPHI_EDGES)  # confirmation

df_edges.head(10)  # aperçu de quelques liens


Saved: data/gephi_nodes.csv
Saved: data/gephi_edges.csv


,Source,Target,Type
0,cupcakesandcashmere.com,www.facebook.com,Directed
1,cupcakesandcashmere.com,twitter.com,Directed
2,cupcakesandcashmere.com,www.instagram.com,Directed
3,cupcakesandcashmere.com,pinterest.com,Directed
4,cupcakesandcashmere.com,www.youtube.com,Directed
5,cupcakesandcashmere.com,www.bloglovin.com,Directed
6,cupcakesandcashmere.com,cupcakesandcashmere.com,Directed
7,cupcakesandcashmere.com,cupcakesandcashmere.com,Directed
8,cupcakesandcashmere.com,cupcakesandcashmere.com,Directed
9,cupcakesandcashmere.com,cupcakesandcashmere.com,Directed


In [15]:
# controle qualité (on verifie si scraping ok)
print("Nb de blogs (seeds) :", len(SEED_BLOGS))  # combien de blogs on a essayé de scraper
print("Nb d'articles :", len(df_articles))  # taille du corpus final
print("Nb de noeuds (domaines) :", len(df_nodes))  # combien de domaines dans le graphe
print("Nb d'arêtes (liens) :", len(df_edges))  # combien de liens dans le graphe

# Voir quels blogs ont fourni le plus d'articles (ça aide pour voir si un blog bug)
df_articles["seed_domain"].value_counts()  # tri automatique (du plus fréquent au moins fréquent)


Nb de blogs (seeds) : 6
Nb d'articles : 16
Nb de noeuds (domaines) : 26
Nb d'arêtes (liens) : 480


seed_domain
www.mindbodygreen.com      15
cupcakesandcashmere.com     1
Name: count, dtype: int64

In [16]:

#debug si un blog ne marche pas
test_seed = SEED_BLOGS[0]  # on prend le premier seed
html = fetch_html(test_seed)  # on télécharge la page d'accueil
print("HTML downloaded:", html is not None)  # vérifier si on a bien du HTML
print("Domain:", get_domain(test_seed))  # vérifier le domaine


HTML downloaded: False
Domain: www.apartmenttherapy.com
